In [ ]:
import nltk

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/olgageta/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/olgageta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import pandas as pd
import os
import re
import unicodedata
from nltk.stem import WordNetLemmatizer
import numpy as np
from collections import defaultdict
from math import log
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pymorphy2

In [9]:
source_dir = 'Data'

In [10]:
#Инициализация объекта MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()

In [11]:
def build_inverted_index(source_dir):
    inverted_index = defaultdict(dict) 
    doc_freq = defaultdict(int) 
    total_docs = 0  

    # Обход файлов в директории
    for filename in os.listdir(source_dir):  
        if filename.endswith(".txt"): 
            total_docs += 1  #счетчиком считаем количество файлов
            file_path = os.path.join(source_dir, filename) 
            
            # чтение файла, преобразование к нижнему регистру,создание токенов
            with open(file_path, 'r') as file:  
                text = file.read().lower()  
                tokens = word_tokenize(text)  
                term_freq = defaultdict(int) 
                
                # для токенов- удаление пунктуации, лемматизация до базовой формы слова
                for token in tokens: 
                    token = re.sub(r'[^\w\s]', '', token) 
                    lemma = morph.parse(token)[0].normal_form  
                    term_freq[lemma] += 1  
                    
                # для термов и частоты их встречания в документе. увеличение частоты при встречании
                for term, freq in term_freq.items(): 
                    inverted_index[term][filename] = freq 
                    doc_freq[term] += 1 
                    

    idf = {term: log(total_docs / freq) for term, freq in doc_freq.items()}  
    
    # обход термов и их частот в документах и подсчет их индекса. с обновление значения и частоты встречания
    for term, doc_freqs in inverted_index.items(): 
        for doc, freq in doc_freqs.items():  
            tfidf = freq * idf[term]
            
            """
            переменная inverted_index прдставляет собой словарь, где ключ - это слово или токен, 
            а значение- в каких документ и скакой частотой эти слова встречаются
            inverted_index[term][doc] = tfidf  

    return inverted_index 

In [12]:
def search_inverted_index(inverted_index, words):
    search_results = defaultdict(float)  
    exact_match_bonus = 2.0 #увеличение веса для точного совпадения словосочетаний в документе

    # обход слов, которые ищем и их лемматизация
    for word in words:  
        lemma = morph.parse(word)[0].normal_form 
        
        # получение документов и их значения tfidf. увеличение счетчика при встречании. 
        for doc, tfidf in inverted_index.get(lemma, {}).items():  
            search_results[doc] += tfidf  
        exact_match = inverted_index.get(lemma, {}).get(lemma)  # проверка точного совпадения словосочетаний
        
        # увеличение значения при точной детекции словосочетания
        if exact_match:
            search_results[lemma] += exact_match * exact_match_bonus 

    return search_results  

In [14]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["симпсон", "устарел"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы, содержащие искомые слова:
postmodern.txt: 6.931471805599452


In [16]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["смена", "постмодерна"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
allmodern.txt: 4.720696194884877
metamodern.txt: 3.4000688682823563
postmodern.txt: 0.28768207245178085


In [19]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["современное", "искусство"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
modernart.txt: 5.753641449035617
metamodern.txt: 0.8630462173553426
allmodern.txt: 0.28768207245178085
postmodern.txt: 0.0


In [20]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["традиционализм", "современность"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
metamodern.txt: 1.6739764335716714
allmodern.txt: 0.28768207245178085
postmodern.txt: 0.28768207245178085


Проверяю функцию, которая приводит искомые слова в исходные формы

In [17]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["рефлексируя", "над", "уже", "созданным"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
allmodern.txt: 9.010913347279288
modernart.txt: 1.3862943611198906
postmodern.txt: 0.6931471805599453


In [18]:
inverted_index = build_inverted_index(source_dir)  

search_words = ["рефлексировать", "над", "уже", "создать"] 
results = search_inverted_index(inverted_index, search_words) 

"""
сортировка результатов по убыванию, чтобы найти документ с наиболее точным совпадением
вывожу score который считается в функции search_inverted_index, он 
подсчитывается путем суммирования tfidf значений для всех слов из запроса поиска
"""

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
allmodern.txt: 9.010913347279288
modernart.txt: 1.3862943611198906
postmodern.txt: 0.6931471805599453
